# Compare Results

This interactive notebook allows you to select completed **MATLAB** and **Python** runs and generate a comparison report.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

# Add project root to path
project_root = Path('..').resolve()
sys.path.insert(0, str(project_root))

from slavv.dev.comparison import run_standalone_comparison

## Select Runs

Choose the output folders from your previous runs.

In [ ]:
comparisons_dir = project_root / "comparisons"
if not comparisons_dir.exists():
    comparisons_dir.mkdir(parents=True, exist_ok=True)

# Find run folders
all_runs = sorted([d.name for d in comparisons_dir.iterdir() if d.is_dir()], reverse=True)

# Heuristic filtering
matlab_candidates = [r for r in all_runs if 'matlab' in r.lower()] + [r for r in all_runs if 'matlab' not in r.lower()]
python_candidates = [r for r in all_runs if 'python' in r.lower()] + [r for r in all_runs if 'python' not in r.lower()]

# Defaults
default_matlab = matlab_candidates[0] if matlab_candidates else None
default_python = python_candidates[0] if python_candidates else None
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
default_output_name = f"{timestamp}_comparison_report"

# Styles
section_header_style = "font-size: 14px; font-weight: bold; color: #2c3e50; margin-bottom: 5px; border-bottom: 2px solid #eaeaea; padding-bottom: 5px; width: 100%"
box_layout = widgets.Layout(
    border='1px solid #e0e0e0', 
    padding='15px', 
    margin='10px 0', 
    border_radius='8px',
    width='95%'
)

def create_header(text):
    return widgets.HTML(f"<div style='{section_header_style}'>{text}</div>")

# Widgets
matlab_widget = widgets.Dropdown(
    options=matlab_candidates,
    value=default_matlab,
    description='MATLAB Run:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='100%')
)

python_widget = widgets.Dropdown(
    options=python_candidates,
    value=default_python,
    description='Python Run:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='100%')
)

output_widget = widgets.Text(
    value=default_output_name,
    description='Output Name:',
    placeholder='Name of the new comparison folder',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='100%')
)


selection_box = widgets.VBox([
    create_header("Select Runs"),
    matlab_widget,
    python_widget,
    create_header("Output"),
    output_widget
], layout=box_layout)

display(selection_box)

## Run Comparison

In [ ]:
run_button = widgets.Button(
    description='Compare Results',
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    layout=widgets.Layout(width='100%', height='50px'),
    icon='balance-scale', 
    style={'font_weight': 'bold', 'font_size': '16px'}
)

output_area = widgets.Output(
    layout=widgets.Layout(
        border='1px solid #ddd',
        padding='10px',
        margin='20px 0',
        max_height='400px',
        overflow='scroll'
    )
)

def on_run_clicked(b):
    # Get values
    matlab_folder = matlab_widget.value
    python_folder = python_widget.value
    output_name = output_widget.value
    
    if not matlab_folder or not python_folder:
        with output_area: print("Error: Please select both runs.")
        return
        
    matlab_path = comparisons_dir / matlab_folder
    python_path = comparisons_dir / python_folder
    output_path = comparisons_dir / output_name

    run_button.disabled = True
    run_button.description = "Comparing..."
    
    with output_area:
        clear_output()
        print(f"Starting comparison at {datetime.now().strftime('%H:%M:%S')}...")
        print("-" * 40)
        
        try:
            status = run_standalone_comparison(
                matlab_dir=matlab_path,
                python_dir=python_path,
                output_dir=output_path,
                project_root=project_root
            )
            
            print("-" * 40)
            if status == 0:
                print("\nComparison report generated successfully!")
            else:
                print("\nComparison failed. Check logs above.")

        except Exception as e:
            print(f"\nAn unexpected error occurred: {e}")
            import traceback
            traceback.print_exc()
        finally:
            run_button.disabled = False
            run_button.description = 'Compare Results'

run_button.on_click(on_run_clicked)

display(widgets.VBox([run_button, output_area]))

## Next Steps

- Open `04_Comparison_Dashboard.ipynb` to analyze the results.
- Check `06_Data_Management.ipynb` to generate a manifest or clean up files.